In [1]:
import os
import json
import pandas as pd
import mysql.connector

In [2]:
connection = mysql.connector.connect(
    host = 'localhost',
    port = '3306',
    user = 'root',
    password = '1234',
    database = 'phonepe'
)
cursor = connection.cursor()

In [56]:
# Aggregated Transaction Data
agg_trans_path = 'C:/Users/welcome/Documents/guvi-projects/phonepe/pulse/data/aggregated/transaction/country/india/state/'
agg_trans_list = os.listdir(agg_trans_path)
agg_trans_columns = {"state": [], "year": [], "quarter": [], "transaction_type": [], "transaction_count": [], "transaction_amount": [],}
for state in agg_trans_list:
    state_url = agg_trans_path+state+'/'
    years = os.listdir(state_url)
    for year in years:
        year_url = state_url+year+'/'
        files = os.listdir(year_url)
        for f in files:
            file_url = year_url+f
            data = open(file_url, 'r')
            agg_trans_data= json.load(data)
            for d in agg_trans_data['data']['transactionData']:
                name = d['name'],
                count = d['paymentInstruments'][0]['count']
                amount = d['paymentInstruments'][0]['amount']
                agg_trans_columns['transaction_type'].append(name)
                agg_trans_columns['transaction_count'].append(count)
                agg_trans_columns['transaction_amount'].append(amount)
                agg_trans_columns['state'].append(state)
                agg_trans_columns['year'].append(year)
                agg_trans_columns['quarter'].append(int(f.strip('.json')))
agg_trans_df = pd.DataFrame(agg_trans_columns)
agg_trans_df['state'] = agg_trans_df['state'].str.replace('-', ' ')
agg_trans_df['state'] = agg_trans_df['state'].str.title()
agg_trans_df['transaction_type'] = agg_trans_df['transaction_type'].apply(lambda x: x[0])

agg_trans_create_query = '''
CREATE TABLE IF NOT EXISTS aggregated_transactions (
States varchar(255),
Year int,
Quarter int,
Transaction_type varchar(255),
Transaction_count bigint,
Transaction_amount bigint
)
'''
cursor.execute(agg_trans_create_query)
connection.commit()
agg_trans_insert_query = '''INSERT INTO aggregated_transactions (States, Year, Quarter, Transaction_type, Transaction_count, Transaction_amount) 
values(%s,%s,%s,%s,%s,%s)'''
for i, row in agg_trans_df.iterrows():
    values = (row['state'],
              row['year'],
              row['quarter'],
              row['transaction_type'],
              row['transaction_count'],
              row['transaction_amount']
              )
    cursor.execute(agg_trans_insert_query, values)
    connection.commit()

In [70]:
#aggregated insurance data
agg_ins_path = 'C:/Users/welcome/Documents/guvi-projects/phonepe/pulse/data/aggregated/insurance/country/india/state/'
agg_ins_list = os.listdir(agg_ins_path)
agg_ins_columns = {"state": [], "year": [], "quarter": [], "insurance_type": [], "insurance_count": [], "insurance_amount": [],}
for state in agg_ins_list:
    state_url = agg_ins_path+state+'/'
    years = os.listdir(state_url)
    for year in years:
        year_url = state_url+year+'/'
        files = os.listdir(year_url)
        for f in files:
            file_url = year_url+f
            data = open(file_url, 'r')
            agg_ins_data= json.load(data)
            for d in agg_ins_data['data']['transactionData']:
                name = d['name'],
                count = d['paymentInstruments'][0]['count']
                amount = d['paymentInstruments'][0]['amount']
                agg_ins_columns['insurance_type'].append(name)
                agg_ins_columns['insurance_count'].append(count)
                agg_ins_columns['insurance_amount'].append(amount)
                agg_ins_columns['state'].append(state)
                agg_ins_columns['year'].append(year)
                agg_ins_columns['quarter'].append(int(f.strip('.json')))
agg_ins_df = pd.DataFrame(agg_ins_columns)
agg_ins_df['state'] = agg_ins_df['state'].str.replace('-', ' ')
agg_ins_df['state'] = agg_ins_df['state'].str.title()
agg_ins_df['insurance_type'] = agg_ins_df['insurance_type'].apply(lambda x: x[0])

agg_ins_create_query = '''
CREATE TABLE IF NOT EXISTS aggregated_insurance (
States varchar(255),
Year int,
Quarter int,
Insurance_type varchar(255),
Insurance_count bigint,
Insurance_amount bigint
)
'''
cursor.execute(agg_ins_create_query)
connection.commit()
agg_ins_insert_query = '''INSERT INTO aggregated_insurance (States, Year, Quarter, Insurance_type, Insurance_count, Insurance_amount) 
values(%s,%s,%s,%s,%s,%s)'''
for i, row in agg_ins_df.iterrows():
    values = (row['state'],
              row['year'],
              row['quarter'],
              row['insurance_type'],
              row['insurance_count'],
              row['insurance_amount']
              )
    cursor.execute(agg_ins_insert_query, values)
    connection.commit()

In [72]:
#aggregated user data
agg_user_path = 'C:/Users/welcome/Documents/guvi-projects/phonepe/pulse/data/aggregated/user/country/india/state/'
agg_user_list = os.listdir(agg_user_path)

agg_user_columns = {
    "state": [],
    "year": [],
    "quarter": [],
    "brand": [],
    "transaction_count": [],
    "percentage": [],
}

for state in agg_user_list:
    state_url = agg_user_path + state + '/'
    years = os.listdir(state_url)
    for year in years:
        year_url = state_url + year + '/'
        files = os.listdir(year_url)
        for file in files:
            file_url = year_url + file
            with open(file_url, 'r') as data:
                agg_user_data = json.load(data)
                try:
                    for i in agg_user_data["data"]["usersByDevice"]:
                        brand = i["brand"]
                        count = i["count"]
                        percentage = i["percentage"]
                        agg_user_columns["brand"].append(brand)
                        agg_user_columns["transaction_count"].append(count)
                        agg_user_columns["percentage"].append(percentage)
                        agg_user_columns["state"].append(state)
                        agg_user_columns["year"].append(year)
                        agg_user_columns["quarter"].append(int(file.strip(".json")))
                except:
                    pass

agg_user_df = pd.DataFrame(agg_user_columns)

agg_user_df['state'] = agg_user_df['state'].str.replace('-', ' ')
agg_user_df['state'] = agg_user_df['state'].str.title()

agg_user_create_query = '''
CREATE TABLE IF NOT EXISTS aggregated_user (
States varchar(255),
Year int,
Quarter int,
Brand varchar(255),
Transaction_count bigint,
Percentage float
)
'''
cursor.execute(agg_user_create_query)
connection.commit()

agg_user_insert_query = '''INSERT INTO aggregated_user (States, Year, Quarter, Brand, Transaction_count, Percentage) 
VALUES(%s, %s, %s, %s, %s, %s)'''

for i, row in agg_user_df.iterrows():
    values = (row['state'],
              row['year'],
              row['quarter'],
              row['brand'],
              row['transaction_count'],
              row['percentage']
              )
    cursor.execute(agg_user_insert_query, values)
    connection.commit()

In [73]:
#map insurance data
map_insurance_path = 'C:/Users/welcome/Documents/guvi-projects/phonepe/pulse/data/map/insurance/hover/country/india/state/'
map_insurance_list = os.listdir(map_insurance_path)

map_insurance_columns = {
    "state": [],
    "year": [],
    "quarter": [],
    "district": [],
    "transaction_count": [],
    "transaction_amount": [],
}

for state in map_insurance_list:
    state_url = map_insurance_path + state + '/'
    years = os.listdir(state_url)
    for year in years:
        year_url = state_url + year + '/'
        files = os.listdir(year_url)
        for file in files:
            file_url = year_url + file
            with open(file_url, 'r') as data:
                map_insurance_data = json.load(data)
                for i in map_insurance_data["data"]["hoverDataList"]:
                    name = i["name"]
                    count = i["metric"][0]["count"]
                    amount = i["metric"][0]["amount"]
                    map_insurance_columns["district"].append(name)
                    map_insurance_columns["transaction_count"].append(count)
                    map_insurance_columns["transaction_amount"].append(amount)
                    map_insurance_columns["state"].append(state)
                    map_insurance_columns["year"].append(year)
                    map_insurance_columns["quarter"].append(int(file.strip(".json")))

map_insurance_df = pd.DataFrame(map_insurance_columns)

map_insurance_df['state'] = map_insurance_df['state'].str.replace('-', ' ')
map_insurance_df['state'] = map_insurance_df['state'].str.title()
map_insurance_df['district'] = map_insurance_df['district'].str.title()

map_insurance_create_query = '''
CREATE TABLE IF NOT EXISTS map_insurance (
State varchar(255),
Year int,
Quarter int,
District varchar(255),
Transaction_count bigint,
Transaction_amount bigint
)
'''
cursor.execute(map_insurance_create_query)
connection.commit()

map_insurance_insert_query = '''INSERT INTO map_insurance (State, Year, Quarter, District, Transaction_count, Transaction_amount) 
VALUES(%s, %s, %s, %s, %s, %s)'''

for i, row in map_insurance_df.iterrows():
    values = (row['state'],
              row['year'],
              row['quarter'],
              row['district'],
              row['transaction_count'],
              row['transaction_amount']
              )
    cursor.execute(map_insurance_insert_query, values)
    connection.commit()


In [74]:
#map transaction data
map_transaction_path = 'C:/Users/welcome/Documents/guvi-projects/phonepe/pulse/data/map/transaction/hover/country/india/state/'
map_transaction_list = os.listdir(map_transaction_path)

map_transaction_columns = {
    "state": [],
    "year": [],
    "quarter": [],
    "district": [],
    "transaction_count": [],
    "transaction_amount": [],
}

for state in map_transaction_list:
    state_url = map_transaction_path + state + '/'
    years = os.listdir(state_url)
    for year in years:
        year_url = state_url + year + '/'
        files = os.listdir(year_url)
        for file in files:
            file_url = year_url + file
            with open(file_url, 'r') as data:
                map_transaction_data = json.load(data)
                for i in map_transaction_data['data']["hoverDataList"]:
                    name = i["name"]
                    count = i["metric"][0]["count"]
                    amount = i["metric"][0]["amount"]
                    map_transaction_columns["district"].append(name)
                    map_transaction_columns["transaction_count"].append(count)
                    map_transaction_columns["transaction_amount"].append(amount)
                    map_transaction_columns["state"].append(state)
                    map_transaction_columns["year"].append(year)
                    map_transaction_columns["quarter"].append(int(file.strip(".json")))

map_transaction_df = pd.DataFrame(map_transaction_columns)

map_transaction_df['state'] = map_transaction_df['state'].str.replace('-', ' ')
map_transaction_df['state'] = map_transaction_df['state'].str.title()
map_transaction_df['district'] = map_transaction_df['district'].str.title()

map_transaction_create_query = '''
CREATE TABLE IF NOT EXISTS map_transaction (
State varchar(255),
Year int,
Quarter int,
District varchar(255),
Transaction_count bigint,
Transaction_amount bigint
)
'''
cursor.execute(map_transaction_create_query)
connection.commit()

map_transaction_insert_query = '''INSERT INTO map_transaction (State, Year, Quarter, District, Transaction_count, Transaction_amount) 
VALUES(%s, %s, %s, %s, %s, %s)'''

for i, row in map_transaction_df.iterrows():
    values = (row['state'],
              row['year'],
              row['quarter'],
              row['district'],
              row['transaction_count'],
              row['transaction_amount']
              )
    cursor.execute(map_transaction_insert_query, values)
    connection.commit()


In [75]:
#map user data
map_user_path = 'C:/Users/welcome/Documents/guvi-projects/phonepe/pulse/data/map/user/hover/country/india/state/'
map_user_list = os.listdir(map_user_path)

map_user_columns = {
    "state": [],
    "year": [],
    "quarter": [],
    "district": [],
    "registered_user": [],
    "app_opens": [],
}

for state in map_user_list:
    state_url = map_user_path + state + '/'
    years = os.listdir(state_url)
    for year in years:
        year_url = state_url + year + '/'
        files = os.listdir(year_url)
        for file in files:
            file_url = year_url + file
            with open(file_url, 'r') as data:
                map_user_data = json.load(data)
                for district, metrics in map_user_data["data"]["hoverData"].items():
                    registered_user = metrics["registeredUsers"]
                    app_opens = metrics["appOpens"]
                    map_user_columns["district"].append(district)
                    map_user_columns["registered_user"].append(registered_user)
                    map_user_columns["app_opens"].append(app_opens)
                    map_user_columns["state"].append(state)
                    map_user_columns["year"].append(year)
                    map_user_columns["quarter"].append(int(file.strip(".json")))

map_user_df = pd.DataFrame(map_user_columns)

map_user_df['state'] = map_user_df['state'].str.replace('-', ' ')
map_user_df['state'] = map_user_df['state'].str.title()
map_user_df['district'] = map_user_df['district'].str.title()

map_user_create_query = '''
CREATE TABLE IF NOT EXISTS map_user (
State varchar(255),
Year int,
Quarter int,
District varchar(255),
Registered_user bigint,
App_opens bigint
)
'''
cursor.execute(map_user_create_query)
connection.commit()

map_user_insert_query = '''INSERT INTO map_user (State, Year, Quarter, District, Registered_user, App_opens) 
VALUES(%s, %s, %s, %s, %s, %s)'''

for i, row in map_user_df.iterrows():
    values = (row['state'],
              row['year'],
              row['quarter'],
              row['district'],
              row['registered_user'],
              row['app_opens']
              )
    cursor.execute(map_user_insert_query, values)
    connection.commit()

In [76]:
#top insurance data
top_insurance_path = 'C:/Users/welcome/Documents/guvi-projects/phonepe/pulse/data/top/insurance/country/india/state/'
top_insurance_list = os.listdir(top_insurance_path)

top_insurance_columns = {
    "state": [],
    "year": [],
    "quarter": [],
    "pincode": [],
    "transaction_count": [],
    "transaction_amount": [],
}

for state in top_insurance_list:
    state_url = top_insurance_path + state + '/'
    years = os.listdir(state_url)
    for year in years:
        year_url = state_url + year + '/'
        files = os.listdir(year_url)
        for file in files:
            file_url = year_url + file
            with open(file_url, 'r') as data:
                top_insurance_data = json.load(data)
                for i in top_insurance_data["data"]["pincodes"]:
                    entityName = i["entityName"]
                    count = i["metric"]["count"]
                    amount = i["metric"]["amount"]
                    top_insurance_columns["pincode"].append(entityName)
                    top_insurance_columns["transaction_count"].append(count)
                    top_insurance_columns["transaction_amount"].append(amount)
                    top_insurance_columns["state"].append(state)
                    top_insurance_columns["year"].append(year)
                    top_insurance_columns["quarter"].append(int(file.strip(".json")))

top_insurance_df = pd.DataFrame(top_insurance_columns)

top_insurance_df['state'] = top_insurance_df['state'].str.replace('-', ' ')
top_insurance_df['state'] = top_insurance_df['state'].str.title()
top_insurance_df['pincode'] = top_insurance_df['pincode'].str.title()

top_insurance_create_query = '''
CREATE TABLE IF NOT EXISTS top_insurance (
State varchar(255),
Year int,
Quarter int,
Pincode varchar(255),
Transaction_count bigint,
Transaction_amount bigint
)
'''
cursor.execute(top_insurance_create_query)
connection.commit()

top_insurance_insert_query = '''INSERT INTO top_insurance (State, Year, Quarter, Pincode, Transaction_count, Transaction_amount) 
VALUES(%s, %s, %s, %s, %s, %s)'''

for i, row in top_insurance_df.iterrows():
    values = (row['state'],
              row['year'],
              row['quarter'],
              row['pincode'],
              row['transaction_count'],
              row['transaction_amount']
              )
    cursor.execute(top_insurance_insert_query, values)
    connection.commit()

In [77]:
#top transaction data
top_transaction_path = 'C:/Users/welcome/Documents/guvi-projects/phonepe/pulse/data/top/transaction/country/india/state/'
top_transaction_list = os.listdir(top_transaction_path)

top_transaction_columns = {
    "state": [],
    "year": [],
    "quarter": [],
    "pincode": [],
    "transaction_count": [],
    "transaction_amount": [],
}

for state in top_transaction_list:
    state_url = top_transaction_path + state + '/'
    years = os.listdir(state_url)
    for year in years:
        year_url = state_url + year + '/'
        files = os.listdir(year_url)
        for file in files:
            file_url = year_url + file
            with open(file_url, 'r') as data:
                top_transaction_data = json.load(data)
                for i in top_transaction_data["data"]["pincodes"]:
                    entityName = i["entityName"]
                    count = i["metric"]["count"]
                    amount = i["metric"]["amount"]
                    top_transaction_columns["pincode"].append(entityName)
                    top_transaction_columns["transaction_count"].append(count)
                    top_transaction_columns["transaction_amount"].append(amount)
                    top_transaction_columns["state"].append(state)
                    top_transaction_columns["year"].append(year)
                    top_transaction_columns["quarter"].append(int(file.strip(".json")))

top_transaction_df = pd.DataFrame(top_transaction_columns)

top_transaction_df['state'] = top_transaction_df['state'].str.replace('-', ' ')
top_transaction_df['state'] = top_transaction_df['state'].str.title()
top_transaction_df['pincode'] = top_transaction_df['pincode'].str.title()

top_transaction_create_query = '''
CREATE TABLE IF NOT EXISTS top_transaction (
State varchar(255),
Year int,
Quarter int,
Pincode varchar(255),
Transaction_count bigint,
Transaction_amount bigint
)
'''
cursor.execute(top_transaction_create_query)
connection.commit()

top_transaction_insert_query = '''INSERT INTO top_transaction (State, Year, Quarter, Pincode, Transaction_count, Transaction_amount) 
VALUES(%s, %s, %s, %s, %s, %s)'''

for i, row in top_transaction_df.iterrows():
    values = (row['state'],
              row['year'],
              row['quarter'],
              row['pincode'],
              row['transaction_count'],
              row['transaction_amount']
              )
    cursor.execute(top_transaction_insert_query, values)
    connection.commit()

In [78]:
#top user data
top_user_path = 'C:/Users/welcome/Documents/guvi-projects/phonepe/pulse/data/top/user/country/india/state/'
top_user_list = os.listdir(top_user_path)

top_user_columns = {
    "state": [],
    "year": [],
    "quarter": [],
    "pincode": [],
    "registered_user": [],
}

for state in top_user_list:
    state_url = top_user_path + state + '/'
    years = os.listdir(state_url)
    for year in years:
        year_url = state_url + year + '/'
        files = os.listdir(year_url)
        for file in files:
            file_url = year_url + file
            with open(file_url, 'r') as data:
                top_user_data = json.load(data)
                for i in top_user_data["data"]["pincodes"]:
                    name = i["name"]
                    registered_users = i["registeredUsers"]
                    top_user_columns["pincode"].append(name)
                    top_user_columns["registered_user"].append(registered_users)
                    top_user_columns["state"].append(state)
                    top_user_columns["year"].append(year)
                    top_user_columns["quarter"].append(int(file.strip(".json")))

top_user_df = pd.DataFrame(top_user_columns)

top_user_df['state'] = top_user_df['state'].str.replace('-', ' ')
top_user_df['state'] = top_user_df['state'].str.title()
top_user_df['pincode'] = top_user_df['pincode'].str.title()

top_user_create_query = '''
CREATE TABLE IF NOT EXISTS top_user (
State varchar(255),
Year int,
Quarter int,
Pincode varchar(255),
Registered_user bigint
)
'''
cursor.execute(top_user_create_query)
connection.commit()

top_user_insert_query = '''INSERT INTO top_user (State, Year, Quarter, Pincode, Registered_user) 
VALUES(%s, %s, %s, %s, %s)'''

for i, row in top_user_df.iterrows():
    values = (row['state'],
              row['year'],
              row['quarter'],
              row['pincode'],
              row['registered_user']
              )
    cursor.execute(top_user_insert_query, values)
    connection.commit()

In [3]:
import mysql.connector
def get_db_connection():
    connection = mysql.connector.connect(
        host = 'localhost',
        port = '3306',
        user = 'root',
        password = '1234',
        database = 'phonepe'
    )
    return connection

def fetch_all_data_from_table(query):
    conn = get_db_connection()
    cursor = conn.cursor(dictionary=True)
    cursor.execute(query)
    results = cursor.fetchall()
    cursor.close()
    conn.close()
    return results

In [4]:
import pandas as pd
table1 = fetch_all_data_from_table("SELECT * FROM aggregated_insurance")
agg_insurance_df = pd.DataFrame(table1)

table2 = fetch_all_data_from_table("SELECT * FROM aggregated_transactions")
agg_transactions_df = pd.DataFrame(table2)

table3 = fetch_all_data_from_table("SELECT * FROM aggregated_user")
agg_user_df = pd.DataFrame(table3)

table4 = fetch_all_data_from_table("SELECT * FROM map_insurance")
map_insurance_df = pd.DataFrame(table4)

table5 = fetch_all_data_from_table("SELECT * FROM map_transaction")
map_transactions_df = pd.DataFrame(table5)

table6 = fetch_all_data_from_table("SELECT * FROM map_user")
map_user_df = pd.DataFrame(table6)

table7 = fetch_all_data_from_table("SELECT * FROM top_insurance")
top_insurance_df = pd.DataFrame(table7)

table8 = fetch_all_data_from_table("SELECT * FROM top_transaction")
top_transactions_df = pd.DataFrame(table8)

table9 = fetch_all_data_from_table("SELECT * FROM top_user")
top_user_df = pd.DataFrame(table9)